In [4]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!pip install geocoder
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

  Using cached https://files.pythonhosted.org/packages/4f/6b/13166c909ad2f2d76b929a4227c952630ebaf0d729f6317eb09cbceccbab/geocoder-1.38.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/f2/98/7e6d147fd16a10a5f821db6e25f192265d6ecca3d82957a4fdd592cad49c/ratelim-0.1.6-py2.py3-none-any.whl
Libraries imported.


In [5]:
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai").text

In [6]:
soup = BeautifulSoup(data, 'html.parser')

In [8]:
neighborhoodList = []

In [9]:
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [10]:
m_df = pd.DataFrame({"Neighborhood": neighborhoodList})

m_df.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


In [11]:
m_df

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup
5,Borivali
6,Charkop
7,Chembur
8,Dahisar
9,Devipada


In [13]:
m_df.shape

(42, 1)

In [14]:
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [16]:
crd = [ get_latlng(neighborhood) for neighborhood in m_df["Neighborhood"].tolist() ]

In [17]:
df_crd = pd.DataFrame(crd, columns=['Latitude', 'Longitude'])

In [18]:
m_df['Latitude'] = df_crd['Latitude']
m_df['Longitude'] = df_crd['Longitude']

In [19]:
m_df.shape

(42, 3)

In [20]:
m_df

,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560
5,Borivali,19.229360,72.857510
6,Charkop,19.208660,72.826120
7,Chembur,19.062180,72.902410
8,Dahisar,19.250030,72.859070
9,Devipada,19.224690,72.866050


In [21]:
m_df.to_csv("m_df.csv", index=False)

In [23]:
address = 'Mumbai'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [28]:
map_mum = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(m_df['Latitude'], m_df['Longitude'], m_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mum)  
    
map_mum

In [29]:
map_mum.save('map_mum.html')

In [31]:
CLIENT_ID = '3120YDYRIBFLCWGUKT13QZVDXC5POMLNUC5N12JO3WTNRR20' # your Foursquare ID
CLIENT_SECRET = 'CZEKK14Q4Y3KAN2FQ4A5TKAPOSAZBQK2NIXWD0KVCGSHPD5Y' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3120YDYRIBFLCWGUKT13QZVDXC5POMLNUC5N12JO3WTNRR20
CLIENT_SECRET:CZEKK14Q4Y3KAN2FQ4A5TKAPOSAZBQK2NIXWD0KVCGSHPD5Y


In [33]:
radius = 500
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(m_df['Latitude'], m_df['Longitude'], m_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [34]:
venues_df = pd.DataFrame(venues)

venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(483, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
2,Andheri,19.118459,72.841763,McDonald's,19.119691,72.846102,Fast Food Restaurant
3,Andheri,19.118459,72.841763,Cafe Alfa,19.119667,72.843560,Indian Restaurant
4,Andheri,19.118459,72.841763,Shoppers Stop,19.114935,72.842708,Department Store


In [35]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,16,16,16,16,16,16
Anushakti Nagar,6,6,6,6,6,6
Baiganwadi,1,1,1,1,1,1
Bandra,15,15,15,15,15,15
Bhandup,5,5,5,5,5,5
Borivali,16,16,16,16,16,16
Charkop,4,4,4,4,4,4
Chembur,23,23,23,23,23,23
Dahisar,6,6,6,6,6,6


In [36]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 111 uniques categories.


In [37]:
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Indian Restaurant', 'Fast Food Restaurant',
       'Department Store', 'Vegetarian / Vegan Restaurant', 'Restaurant',
       'Athletics & Sports', 'Juice Bar', 'Shop & Service', 'Mountain',
       'Food Court', 'Shopping Mall', 'Chinese Restaurant',
       'Paper / Office Supplies Store', 'Sandwich Place', 'Bus Station',
       'Snack Place', 'Convenience Store', 'Arcade', 'Gym', 'ATM', 'Café',
       'Train Station', 'Italian Restaurant', 'Platform', 'Pier',
       'Furniture / Home Store', 'Burger Joint', 'Asian Restaurant',
       'Pharmacy', 'Food Truck', 'Ice Cream Shop', 'Clothing Store',
       'Pizza Place', 'Playground', 'Motorcycle Shop', 'Bar', 'Garden',
       'Lounge', 'Seafood Restaurant', 'Gym / Fitness Center',
       'South Indian Restaurant', 'Accessories Store', 'Market',
       'Sporting Goods Shop', 'Dessert Shop', 'Auto Garage', 'Donut Shop',
       'Coffee Shop', 'Metro Station'], dtype=object)

In [38]:
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [39]:
mum_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

mum_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

fixed_columns = [mum_onehot.columns[-1]] + list(mum_onehot.columns[:-1])
mum_onehot = mum_onehot[fixed_columns]

print(mum_onehot.shape)
mum_onehot.head()

(483, 112)


,Neighborhoods,ATM,Accessories Store,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bar,Basketball Court,Beach,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Bus Station,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Creperie,Cupcake Shop,Dance Studio,Department Store,Design Studio,Dessert Shop,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Market,Men's Store,Metro Station,Mobile Phone Shop,Moroccan Restaurant,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multiplex,Music Venue,Neighborhood,Nightclub,Paper / Office Supplies Store,Park,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pub,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Tea Room,Temple,Thai Restaurant,Theme Park,Train Station,Vegetarian / Vegan Restaurant,Women's Store
0,Andheri,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [40]:
mum_grouped = mum_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mum_grouped.shape)
mum_grouped

(39, 112)


,Neighborhoods,ATM,Accessories Store,Antique Shop,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bar,Basketball Court,Beach,Bistro,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Burger Joint,Bus Station,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Convenience Store,Creperie,Cupcake Shop,Dance Studio,Department Store,Design Studio,Dessert Shop,Diner,Donut Shop,Electronics Store,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,General College & University,General Entertainment,Gift Shop,Grocery Store,Gym,Gym / Fitness Center,Hotel,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Japanese Restaurant,Juice Bar,Liquor Store,Lounge,Market,Men's Store,Metro Station,Mobile Phone Shop,Moroccan Restaurant,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multiplex,Music Venue,Neighborhood,Nightclub,Paper / Office Supplies Store,Park,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pub,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Tea Room,Temple,Thai Restaurant,Theme Park,Train Station,Vegetarian / Vegan Restaurant,Women's Store
0,Andheri,0.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.062500,0.0,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.062500,0.000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500,0.00,0.00,0.000000,0.000000,0.062500,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125000,0.000000,0.00,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.0625,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.062500,0.000000,0.062500,0.000,0.000000,0.0625,0.062500,0.000,0.000000,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.062500,0.000000
1,Anushakti Nagar,0.0,0.000000,0.000000,0.166667,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.166667,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.00,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.000,0.166667,0.000000,0.000000,0.00,0.0,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.000000,0.000000,0.000000
2,Baiganwadi,1.0,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.0000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [41]:
len(mum_grouped[mum_grouped["Indian Restaurant"] > 0])

24

In [42]:
mum_rest = mum_grouped[["Neighborhoods" , "Indian Restaurant"]]

In [43]:
mum_rest.head()

,Neighborhoods,Indian Restaurant
0,Andheri,0.125000
1,Anushakti Nagar,0.000000
2,Baiganwadi,0.000000
3,Bandra,0.333333
4,Bhandup,0.000000


In [45]:
kclusters = 3

mum_clustering = mum_rest.drop(["Neighborhoods"], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mum_clustering)

kmeans.labels_[0:10]

array([0, 1, 1, 2, 1, 1, 0, 0, 0, 1])

In [46]:
mum_merged = mum_rest.copy()
mum_merged["Cluster Labels"] = kmeans.labels_

In [47]:
mum_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mum_merged.head()

,Neighborhood,Indian Restaurant,Cluster Labels
0,Andheri,0.125000,0
1,Anushakti Nagar,0.000000,1
2,Baiganwadi,0.000000,1
3,Bandra,0.333333,2
4,Bhandup,0.000000,1


In [48]:
mum_merged = mum_merged.join(m_df.set_index("Neighborhood"), on="Neighborhood")
print(mum_merged.shape)

(39, 5)


In [49]:
mum_merged.head()

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
0,Andheri,0.125000,0,19.118459,72.841763
1,Anushakti Nagar,0.000000,1,19.042830,72.927340
2,Baiganwadi,0.000000,1,19.062940,72.926630
3,Bandra,0.333333,2,19.054370,72.840170
4,Bhandup,0.000000,1,19.145560,72.948560


In [50]:
print(mum_merged.shape)
mum_merged.sort_values(["Cluster Labels"], inplace=True)
mum_merged

(39, 5)


,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
0,Andheri,0.125000,0,19.118459,72.841763
24,"Matharpacady, Mumbai",0.250000,0,18.950694,72.827268
16,Juhu,0.277778,0,19.014920,72.845220
15,Jogeshwari,0.166667,0,19.137920,72.849410
31,"Sonapur, Bhandup",0.200000,0,19.163940,72.935440
6,Charkop,0.250000,0,19.208660,72.826120
7,Chembur,0.217391,0,19.062180,72.902410
8,Dahisar,0.166667,0,19.250030,72.859070
30,"Sion, Mumbai",0.200000,0,19.043410,72.863320
25,Mira Road,0.142857,0,19.265674,72.870681


In [51]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mum_merged['Latitude'], mum_merged['Longitude'], mum_merged['Neighborhood'], mum_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [52]:
map_clusters.save('map_clusters.html')

### Cluster 1

In [53]:
mum_merged.loc[mum_merged['Cluster Labels'] == 0]

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
0,Andheri,0.125000,0,19.118459,72.841763
24,"Matharpacady, Mumbai",0.250000,0,18.950694,72.827268
16,Juhu,0.277778,0,19.014920,72.845220
15,Jogeshwari,0.166667,0,19.137920,72.849410
31,"Sonapur, Bhandup",0.200000,0,19.163940,72.935440
6,Charkop,0.250000,0,19.208660,72.826120
7,Chembur,0.217391,0,19.062180,72.902410
8,Dahisar,0.166667,0,19.250030,72.859070
30,"Sion, Mumbai",0.200000,0,19.043410,72.863320
25,Mira Road,0.142857,0,19.265674,72.870681


### Cluster 2

In [54]:
mum_merged.loc[mum_merged['Cluster Labels'] == 1]

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
23,Mankhurd,0.000000,1,19.048530,72.932220
26,Mulund,0.100000,1,19.171830,72.955650
29,Seven Bungalows,0.057143,1,19.131342,72.816342
22,Mahavir Nagar (Kandivali),0.086957,1,19.210940,72.841370
32,Thakur village,0.103448,1,19.210200,72.875410
33,Tilak Nagar (Mumbai),0.000000,1,18.996160,72.852790
35,Vikhroli,0.000000,1,19.111090,72.927810
36,Wadala,0.000000,1,19.017200,72.858170
28,Pestom sagar,0.111111,1,19.070640,72.902170
21,Kurla,0.000000,1,19.064980,72.880690


### Cluster 3

In [56]:
mum_merged.loc[mum_merged['Cluster Labels'] == 2]

,Neighborhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
27,Mumbra,0.500000,2,19.167632,73.021408
13,Goregaon,0.352941,2,19.164550,72.849460
14,Grant Road,0.384615,2,18.959290,72.831080
3,Bandra,0.333333,2,19.054370,72.840170
34,Vashi,0.357143,2,19.084650,72.904810
